In [ ]:
#정형 데이터의 끝판왕 앙상블 학습 알고리즘
#Random Forest: 입력한 훈련 데이터에서 랜덤하게 샘플을 추출하여 훈련 데이터를 만듦
#생성된 훈련 데이터: 부트스트랩 샘플(샘플 중복 추출 가능)
#각 노드를 분할할 대 전체 특성 중 일부 특성을 무작위로 고르고 최선의 분할을 찾음
#RandomForestClassifier는 기본적으로 전체 특성 개수의 제곱근만큼의 특성 선택
#RandomForestRegressor는 전체 특성 사용

In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol', 'sugar', 'pH']]
target = wine['class']

train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size = 0.2, random_state= 42)

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs = -1, random_state=42)
scores = cross_validate(rf, train_input, train_target,
                        return_train_score=True, n_jobs = -1)
#cross_validate: 사이킷런 교차검증 함수, 훈련과 평가를 여러번 반복하여 성능 점수 계산

print(np.mean(scores['train_score']), np.mean(scores['test_score']))
#과대적합

0.9973541965122431 0.8905151032797809


In [ ]:
rf.fit(train_input, train_target)
print(rf.feature_importances_) 
#결정트리의 중요도와 비교했을 때 변수 중요도가 달라짐
#특성의 일부를 랜덤하게 선택하여 결정트리를 훈련했기때문에 하나의 특성에 과도하게 집중하지 않고 더 많은 특성으로 훈련할 기회가 있었음, 즉 일반화 성능 증가


[0.23167441 0.50039841 0.26792718]


In [ ]:
#부트스트랩에 포함되지 않고 남은 샘플 : OBB(out of bag) sample
#OOB sample로 부트스트랩 샘플로 훈련한 결정트리 모델을 평가, 마치 검증세트같은
#Oob_score  매개변수 활용 기본값은 false, True로 지정 필요

rf = RandomForestClassifier(oob_score= True, n_jobs=-1, random_state=42)
#n_jobs : cpu core 사용 개수, -1(최대 병렬 처리, 모든 cpu core 사용), 1(단일코어 순차처리, 1개 사용)

rf.fit(train_input, train_target)
print(rf.oob_score_)  #교차검증(cross_validate)을 하지 않고도 결정트리 평가가능
#이처럼 oob 점수 사용시 교차검증을 대신할 수 있기에 결과적으로 훈련세트에 더 많은 샘플을 사용할 수 있음

0.8934000384837406


In [ ]:
#Extra tree: 부트스트랩 샘플 사용하지 않고 결정 트리 만들 때 전체 훈련세트 사용
#노드를 분할할 때, 가장 좋은 분할을 찾는 것이 아닌 무작위로 분할
#DecisionTreeClassifier의 splitter=random으로한 결정트리와 같음

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target,
                        return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

#특성의 개수가 많지 않아서 두 모델 차이가 크지 않음

0.9974503966084433 0.8887848893166506


In [27]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


In [ ]:
#Gradient boosting: 깊이가 얕은 결정트리를 사용하여 이전 트리의 오차 보완하는 앙상블 기법
#경사하강법을 사용하여 트리를 앙상블에 추가함
#이전 트리의 예측 오류(잔차)를 경사(Gradient)로 계산하고, 
#이 오류를 가장 잘 예측하는 새로운 결정 트리를 순차적으로 계속 추가하여
# 모델의 성능을 점진적으로 개선하는 앙상블 기법

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, 
                        return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))
#과대적합을 낮췄음

0.8881086892152563 0.8720430147331015


In [ ]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2,
                                random_state=42)
#결정트리의 개수를 500개(배)로 늘림
scores = cross_validate(gb, train_input, train_target, 
                        return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [33]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15882696 0.6799705  0.16120254]


In [ ]:
#subsampple: 트리 훈련에 사용할 훈련세트의 비율을 정함, 기본값 1.0(전체), 1.0 미만은 훈련세트 일부 사용
#단계마다 일부 샘플 랜덤하게 선택하여 사용하는 경사하강법, 미니배치 경사하강법과 비슷
#그레이디언트 부스팅이 랜덤 포레스트보다 조금 더 높은 성능을 얻을 수 있으나 순서대로 트리 추가하여 느림
#즉 n_jobs 매개변수 없음

In [35]:
#histogram-based gradient boosting: 정형 데이터를 다루는 머신러닝중에 인기 가장 높음
#HistGradientBoostingClassifier,,트리 개수 지정에 n_estimators 대신 부스팅 반복 횟수를 지정하는 max_iter 사용

from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target,
                        return_train_score= True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


In [37]:
from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target,
                                n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


In [38]:
result = permutation_importance(hgb, test_input, test_target,
                                n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [43]:
hgb.score(test_input, test_target)

0.8723076923076923

In [49]:
from xgboost import XGBClassifier
xgb = XGBClassifier(tree_method = 'hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target,
                        return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9567059184812372 0.8783915747390243


In [55]:
from lightgbm import LGBMClassifier
lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target,
                        return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.935828414851749 0.8801251203079884
